In [1]:

# set working directory
import os
os.chdir("/Users/antonwiklund/Documents/Understanding-Tool-Aware-AI-Agents/code/tool-aware-ai-testing")

from src.database.connection import get_connection
from tabulate import tabulate
import pandas as pd


In [7]:
conn, cur = get_connection()

cur.execute("SELECT * FROM results")
results = cur.fetchall()

In [8]:
# print the result pretty
for result in results:
    print(result)


(7, 1, 6, ['statistical_analysis_tool', 'statistical_analysis_tool'], 0.0004704999882960692, True, None, datetime.datetime(2024, 11, 30, 20, 12, 28, 633874))
(8, 2, 6, ['calendar_tool', 'calendar_tool'], 3.533299604896456e-05, False, None, datetime.datetime(2024, 11, 30, 20, 12, 31, 445438))
(9, 3, 6, ['database_tool', 'database_tool'], 0.00014795898459851742, True, None, datetime.datetime(2024, 11, 30, 20, 12, 34, 640078))
(10, 4, 6, ['search_web_tool'], 1.6249992768280208e-05, True, None, datetime.datetime(2024, 11, 30, 20, 12, 36, 889667))
(11, 6, 6, ['statistical_analysis_tool', 'statistical_analysis_tool'], 0.0003815829986706376, True, None, datetime.datetime(2024, 11, 30, 20, 12, 39, 867452))


In [2]:
conn, cur = get_connection()


In [3]:
cur.execute("""
    SELECT 
        r.id,
        p.prompt,
        t.model_name,
        r.tool_calls,
        r.time_taken,
        r.success_rate,
        r.error_type,
        r.created_at
    FROM results r
    JOIN prompts p ON r.prompt_id = p.id
    JOIN test_runs t ON r.test_run_id = t.id
    ORDER BY r.created_at DESC;
""")
results = cur.fetchall()


In [4]:
# Convert to pandas DataFrame with column names
df = pd.DataFrame(results, columns=[
    'ID', 'Prompt', 'Model', 'Tool Calls', 
    'Time Taken', 'Success', 'Error', 'Created At'
])

In [5]:
# Display the results as a formatted table
print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))


+------+------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+------------------------------------------------------------+--------------+-----------+---------+----------------------------+
|   ID | Prompt                                                                                                                                               | Model       | Tool Calls                                                 |   Time Taken | Success   | Error   | Created At                 |
|------+------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+------------------------------------------------------------+--------------+-----------+---------+----------------------------|
|   11 | Generate a statistical report for the following data: [{"duration": 30, "participants": 

In [6]:
# Calculate and display statistics
print("\n=== Test Run Statistics ===")

# Success rate by model
model_stats = df.groupby('Model').agg({
    'Success': ['count', 'mean'],
    'Time Taken': ['mean', 'std']
}).round(3)

print("\nSuccess Rate by Model:")
print(tabulate(model_stats, headers=[
    'Total Tests', 'Success Rate', 'Avg Time', 'Std Time'
], tablefmt='psql'))

# Tool usage statistics
print("\nTool Usage Statistics:")
tool_counts = {}
for tools in df['Tool Calls']:
    for tool in tools:
        tool_counts[tool] = tool_counts.get(tool, 0) + 1

tool_stats = pd.DataFrame.from_dict(
    tool_counts, 
    orient='index', 
    columns=['Count']
).sort_values('Count', ascending=False)

print(tabulate(tool_stats, headers=['Tool', 'Usage Count'], tablefmt='psql'))


=== Test Run Statistics ===

Success Rate by Model:
+-------------+---------------+----------------+------------+------------+
|             |   Total Tests |   Success Rate |   Avg Time |   Std Time |
|-------------+---------------+----------------+------------+------------|
| gpt-4o-mini |             5 |            0.8 |          0 |          0 |
+-------------+---------------+----------------+------------+------------+

Tool Usage Statistics:
+---------------------------+---------------+
| Tool                      |   Usage Count |
|---------------------------+---------------|
| statistical_analysis_tool |             4 |
| database_tool             |             2 |
| calendar_tool             |             2 |
| search_web_tool           |             1 |
+---------------------------+---------------+
